In [4]:
import lightning as L
import torch.nn.functional as F
import timm
import torch

# Define the teacher model

In [ ]:
timm.list_models()

In [ ]:
class TeacherResNet50(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.resnet50 = timm.create_model('restnet50', pretrained=True)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
teacherResnet50 = TeacherResNet50()

# Define the student model

In [ ]:
# Define the 